In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 500)

import re

In [3]:
import numpy as np

In [62]:
df = pd.read_csv("ll_info_dirty.csv")

In [63]:
#df.head()

In [64]:
#df.dtypes

In [65]:
#df.shape

In [66]:
df["group"] = df["  group"]

In [67]:
df = df.drop(columns = "  group")

In [68]:
#df.name.value_counts()

In [69]:
#df[df.name == "王宁"]

### Many leaders hold multiple local position, so I hesitate dropping duplicates ... But it make sense to drop it for this project

In [70]:
df.drop_duplicates(subset=["url"], keep="first", inplace = True)

In [71]:
#df.name.value_counts()

In [72]:
#df[df.name.isin(["刘强", "王宁","李毅","刘惠","陈飞","刘伟","杨斌","陈平","杨军","罗强","王少峰","王建军","李刚","李伟","李清"])]
# Well some names are just more popular than others

In [73]:
#df.shape

### Get gender

In [74]:
df["short_bio"] = df["short_bio"].str.strip()

In [75]:
df["gender"] = df["short_bio"].str.extract(r"([男女])")

In [76]:
#df[df.gender.isnull()].shape

In [77]:
#df.gender.value_counts()

### Get ethnic background


In [78]:
df["ethnic"] = df["short_bio"].str.extract(r"([^，。]{1,}族)")

In [79]:
#df["ethnic"].value_counts()

In [80]:
#df[df.ethnic.isnull()].shape

### Get date of birth

#### dob_y = year of birth
#### age = 2018 - dob_y

In [81]:
def get_dob(row):
    if "生" in row["short_bio"] and "年" in row["short_bio"]:
        if "生于\d" in row["short_bio"]:
            dob_y = re.findall("生于(\d\d\d\d)年", row["short_bio"])
            dob_y = ",".join(dob_y).split(",")[0] 
        else:
            dob_y = re.findall("(\d+)年[^，。]{0,}月", row["short_bio"])
            dob_y = ",".join(dob_y).split(",")[0] 
    else:
        dob_y = "unknown"
    return dob_y

In [82]:
df["dob_y"] = df.apply(get_dob, axis=1)

In [83]:
#df["dob_y"].value_counts()

In [84]:
#df[df.dob_y == ""]

In [85]:
df.at[218, "dob_y"] = 1961
df.at[380, "dob_y"] = 1963
df.at[650, "dob_y"] = 1966

In [86]:
#df[df.dob_y == "unknown"]

In [87]:
df.dob_y.replace("unknown", np.nan, inplace = True)

In [88]:
df.dob_y = df.dob_y.astype(float)

In [89]:
df["age"] = 2018 - df["dob_y"]

### Get when they joined workforce

#### join_work_y = year they joined workforce
#### working_y = 2018 - join_work_y

In [90]:
df["join_work_y"] = df["short_bio"].str.extract(r"(\d{4})年[\d月]{0,3}参加工作[，。,（]").astype(float)

In [91]:
#df[(df["join_work_y"].isnull()) & (df["short_bio"].str.contains("参加工作"))]

In [92]:
df.at[380, "join_work_y"] = 1985
df.at[473, "join_work_y"] = 1985
df.at[785, "join_work_y"] = 1981

### Get place of birth
#### pob = place of birth
#### pob_p = province of birth

In [93]:
def get_pob(row):
    if "籍贯、" in row["short_bio"]:
        pob = re.findall("出生地([^，。]{2,}[,，。])", row["short_bio"])
        pob = "".join(pob)
    elif "籍贯" in row["short_bio"]:
        pob = re.findall("籍贯([^，。]{2,}[,，。])", row["short_bio"])
        pob = "".join(pob)
    elif "出生于" in row["short_bio"]:
        pob = re.findall("出生于([^，。]{2,}[,，。])", row["short_bio"])
        pob = "".join(pob)
    else:
        pob = re.findall("，([^，]{2,})人[，。]", row["short_bio"])
        pob = "".join(pob)
    return pob

In [94]:
df["pob"] = df.apply(get_pob, axis=1)

In [95]:
#df[df["pob"] == ""]

In [96]:
df.pob.replace("", np.nan, inplace = True)

In [97]:
df["pob_p"] = df["pob"].str[0:2]

In [98]:
#df[df["pob_p"].isnull()]

In [99]:
df.at[1576, "pob_p"] = "澳门"
df.at[1576, "pob"] = "澳门"

In [100]:
#df.pob_p.value_counts()

In [101]:
df.pob_p = df.pob_p.str.replace("献县","河北")
df.pob_p = df.pob_p.str.replace("巨鹿","河北")
df.pob_p = df.pob_p.str.replace("衡水","河北")
df.pob_p = df.pob_p.str.replace("安微","安徽")
df.pob_p = df.pob_p.str.replace("玉田","河北")
df.pob_p = df.pob_p.str.replace("麻栗","云南")
df.pob_p = df.pob_p.str.replace("正定","河北")
df.pob_p = df.pob_p.str.replace("衡阳","湖南")
df.pob_p = df.pob_p.str.replace("深泽","河北")
df.pob_p = df.pob_p.str.replace("梅县","广东")
df.pob_p = df.pob_p.str.replace("邢台","河北")
df.pob_p = df.pob_p.str.replace("平山","河北")
df.pob_p = df.pob_p.str.replace("无极","河北")
df.pob_p = df.pob_p.str.replace("威县","河北")
df.pob_p = df.pob_p.str.replace("邯郸","河北")
df.pob_p = df.pob_p.str.replace("京山","湖北")

In [102]:
#df[df.pob_p.isnull()].shape

### Get when they joined CCP
#### join_party_y

In [103]:
df["join_party_y"] = df["short_bio"].str.extract(r"(\d{4})年[\d月]{0,}[加入]{1,}[中国共产党]{1,}[，。]").astype(float)

In [104]:
# df.join_party_y.value_counts()

### Get education background
#### edu_dg = their highest degree
#### edu_am = alma mater + major
#### edu_uni = alma mater
#### edu_major = major

In [105]:
def get_edu_dg(row):
    if "博士" in row["short_bio"]:
        dg = "博士"
    elif "研究生" in row["short_bio"] or "硕士" in row["short_bio"]:
        dg = "研究生"
    elif "大学" in row["short_bio"] or "本科" in row["short_bio"] or "学士" in row["short_bio"] or "大学" in row["short_bio"]:
        dg = "本科"
    elif "大专" in row["short_bio"] or "专科" in row["short_bio"]:
        dg = "大专"
    else:
        dg = "unknown"
    return dg

In [106]:
df["edu_dg"] = df.apply(get_edu_dg, axis=1)

In [107]:
def get_edu_am(row):
    if "毕业于" in row["short_bio"]:
        am = re.findall("毕业于([^，。]{1,})[，。]", row["short_bio"])
        am = "".join(am)
    elif "(" in row["short_bio"] and ")" in row["short_bio"]:
        am = re.findall("\((.*)\)", row["short_bio"])
        am = "".join(am)
    elif "（" in row["short_bio"] and "）" in row["short_bio"]:
        am = re.findall("（(.*)）", row["short_bio"])
        am = "".join(am)
    elif "毕业" in row["short_bio"]:
        am = re.findall("[，。]([^，。]{1,})毕业", row["short_bio"])
        am = "".join(am)
    else:
        am = "unknown"
    return am

In [108]:
df["edu_am"] = df.apply(get_edu_am, axis=1)

In [109]:
def get_edu_uni(row):
    
    if "中央党校" in row["edu_am"]:
        university = "中央党校"
    elif "大学" in row["edu_am"]:
        university = re.findall("([^，]{1,}大学).*", row["edu_am"])
        university = "".join(university)
    elif "研究所" in row["edu_am"]:
        university = re.findall("([^，]{1,}研究所).*", row["edu_am"])
        university = "".join(university)
    elif "学院" in row["edu_am"]:
        university = re.findall("([^，]{1,}学院).*", row["edu_am"])
        university = "".join(university)
    elif "学校" in row["edu_am"]:
        university = re.findall("([^，]{1,}学校).*", row["edu_am"])
        university = "".join(university)
    else:
        university = row["edu_am"]

    return university

In [110]:
df["edu_uni"] = df.apply(get_edu_uni, axis=1)

In [111]:
df.drop(columns = ["short_bio", "pob", "edu_am", "edu_uni", "title", "publish_date", "group"], inplace = True)

In [113]:
df.dtypes

name             object
province         object
url              object
gender           object
ethnic           object
dob_y           float64
age             float64
join_work_y     float64
pob_p            object
join_party_y    float64
edu_dg           object
dtype: object

In [114]:
df.to_csv("ll_info_clean.csv", index = False)

In [403]:
#df_test = pd.read_csv("ll_info_clean.csv")

In [404]:
#df_test.head()